In [84]:
import math
import os
import random
import sys

import matplotlib
import matplotlib.pyplot as plt
import mrcnn.model as modellib
import numpy as np
import skimage.io
from mrcnn import utils, visualize

import coco
import parameters
from imantics import Polygons, Mask
import cv2



# Directory to save logs and trained model
MODEL_DIR = os.path.join(parameters.RESULTS_PATH, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(parameters.MODEL_PATH, "mask_rcnn_coco.h5")


class InferenceConfig(coco.CocoConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
config.display()

# Create model object in inference mode.
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

# Load weights trained on MS-COCO
model.load_weights(COCO_MODEL_PATH, by_name=True)


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                93
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE         

In [56]:
def plot_cars(image,points,alpha=0.1):

    output_image = image.copy()

    for contours in points:
        contours = np.array(contours)
        #print(contours)
        image = cv2.fillPoly(image, pts =[contours], color=parameters.COLOR_BLUE)
        image = cv2.polylines(image, pts =[contours], isClosed=True, color=parameters.COLOR_EDGES, thickness=2)
    output_image = cv2.addWeighted(image, alpha, output_image, 1 - alpha,0, output_image)

    return output_image

In [94]:
def get_car_points(rgb_image):

    results = model.detect([rgb_image], verbose=0)
    r = results[0]
    car_points = []

    for i in range(r['rois'].shape[0]):
        if class_names[r['class_ids'][i]] in parameters.CLASS_NAMES:
            mask_image = np.array(r['masks'][:, :, i]*1.0)
            polygons = Mask(mask_image).polygons()
            car_points.append(polygons.points[0])
    return car_points

In [101]:
bgr_image = cv2.imread(parameters.DATA_PATH+'espacio-libre.jpg')
rgb_image = cv2.cvtColor(bgr_image,cv2.COLOR_BGR2RGB)

car_points = get_car_points(rgb_image)
        
bgr_image = plot_cars(bgr_image,car_points,alpha=0.4)

cv2.imshow('soel',bgr_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [99]:
import cv2
import numpy as np

# Create a VideoCapture object and read from input file
# If the input is the camera, pass 0 instead of the video file name
cap = cv2.VideoCapture(parameters.DATA_PATH+'stace_park3.mp4')

# Check if camera opened successfully
if (cap.isOpened()== False): 
  print("Error opening video stream or file")

# Read until video is completed
while(cap.isOpened()):
  # Capture frame-by-frame
  ret, frame = cap.read()
  if ret == True:

    #frame = cv2.resize(frame,(300,300))
    # Display the resulting frame
    rgb_image = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
    car_points = get_car_points(rgb_image)
            
    frame = plot_cars(frame,car_points,alpha=0.4)

    cv2.imshow('Frame',frame)

    # Press Q on keyboard to  exit
    if cv2.waitKey(20) & 0xFF == ord('q'):
      break

  # Break the loop
  else: 
    break

# When everything done, release the video capture object
cap.release()

# Closes all the frames
cv2.destroyAllWindows()